<a href="https://colab.research.google.com/github/AmitAaranya/ASL-Recognition-with-CNN-PyTorch/blob/main/google-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clone GitHub repo
**URL:** https://github.com/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch

In [1]:
!git clone https://github.com/AmitAaranya/ASL-Recognition-with-Deep-Learning-PyTorch.git ./code/

Cloning into './code'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 71 (delta 24), reused 50 (delta 12), pack-reused 0
Receiving objects: 100% (71/71), 1.79 MiB | 24.50 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [39]:
!git config --global user.email "amitaaranya@gmail.com"
!git config --global user.name "AMIT"

In [40]:
!git --git-dir=./code/.git commit -m "Model save"

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	deleted:    .gitignore
	deleted:    README.md
	deleted:    asl-image-labels.png
	deleted:    data-set-distribution.png
	deleted:    data_loader.py
	deleted:    google-colab.ipynb
	deleted:    notebook.ipynb
	deleted:    requirements.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.config/
	asl_cnn_model.pth
	code/
	datasets/
	sample_data/
	synthetic-asl-alphabet.zip

no changes added to commit (use "git add" and/or "git commit -a")


In [3]:
## Add the folder to system path, so we can import file directly
import sys
sys.path.append('/content/code')

In [ ]:
!pip install -r code/requirements.txt > /dev/null

In [8]:
!pip install kaggle

# Load the ASL Image Data
**URL:** https://www.kaggle.com/datasets/lexset/synthetic-asl-alphabet


In [10]:
!echo "{\"username\":\"amitaaranya\",\"key\":\"e8845441bb41bfd733339ef7c8f50fa4\"}" > /root/.kaggle/kaggle.json

In [11]:
!kaggle datasets download -d lexset/synthetic-asl-alphabet

100% 6.58G/6.58G [06:18<00:00, 22.0MB/s]
100% 6.58G/6.58G [06:18<00:00, 18.7MB/s]


In [12]:
!mkdir datasets

In [13]:
!unzip synthetic-asl-alphabet.zip -d ./datasets/ > /dev/null

# Exploratory Data Analysis(EDA)

In [14]:
# Imports
import torch
import numpy as np

## Load Data

In [15]:
from data_loader import load_datasets

size = 100
x_train, y_train = load_datasets(size = size)
x_test,  y_test = load_datasets(train_or_test_folder="Test_Alphabet",size = size)

print(f"Train dataset count: {len(x_train)} \nTest dataset count: {len(x_test)} ")

Loading datasets:  A B Blank C D E F G H I J K L M N O P Q R S T U V W X Y Z 

Loading datasets:  A B Blank C D E F G H I J K L M N O P Q R S T U V W X Y Z 

Train dataset count: 24300 
Test dataset count: 2700 


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [20]:
from data_loader import alphabet_to_num

def one_hot_coding(unique_labels, data):
    y_oh = torch.zeros(len(data),unique_labels)
    for i, class_word in enumerate(data):
        index = alphabet_to_num(class_word)
        y_oh[i][index] = 1

    return y_oh
y_train,y_test = np.array(y_train),np.array(y_test)
y_train_oh = one_hot_coding(len(np.unique(y_train)),y_train)
y_test_oh = one_hot_coding(len(np.unique(y_train)),y_test)

In [21]:
x_train.shape,y_train.shape

(torch.Size([24300, 3, 100, 100]), (24300,))

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=27, kernel_size=5, padding=2)
        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=4, stride=4)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=27, out_channels=27, kernel_size=5, padding=2)
        # Another max pooling layer
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer
        self.fc1 = nn.Linear(27 * 12 * 12, 27)  # 3x3 is the size after two max pooling layers

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 27 * 12 * 12)  # Flatten the output of conv2 layer
        x = self.fc1(x)
        return x

# Instantiate the model
model = ConvNet()

# Print the model architecture
print(model)

ConvNet(
  (conv1): Conv2d(3, 27, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(27, 27, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3888, out_features=27, bias=True)
)


In [55]:
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(),lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.8)
batch_size = 32

x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_oh_tensor = torch.tensor(y_train_oh, dtype=torch.float32).to(device)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
y_test_oh_tensor = torch.tensor(y_test_oh, dtype=torch.float32).to(device)

train_loader = DataLoader(TensorDataset(x_train_tensor,y_train_oh_tensor),
                          batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_tensor,y_test_oh_tensor),
                          batch_size=batch_size)

<ipython-input-55-8b64149be4f9>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
<ipython-input-55-8b64149be4f9>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_oh_tensor = torch.tensor(y_train_oh, dtype=torch.float32).to(device)
<ipython-input-55-8b64149be4f9>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_tensor = torch.tensor(x_test, dtype=torch.float32).to(device)
<ipython-input-55-8b64149be4f9>:14: UserWarning: To copy construct from a tensor,

In [51]:
len(train_loader.dataset),len(test_loader.dataset)

(24300, 2700)

In [52]:
print(train_loader.batch_size)

32


## Define CNN model

In [ ]:
def train_model(model, criterion, optimizer, device, num_epochs=100):
    model.to(device)  # Move the model to the device outside the loop
    for epoch in range(num_epochs):
        running_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move inputs and labels to device
            # print(f"Inputs size from train_loader: {inputs.size()}")  # Print sizes for debugging
            # print(f"Labels size from train_loader: {labels.size()}")
            optimizer.zero_grad()
            outputs = model(inputs)
            # print(f"Output Tensor size from Model: {outputs.size()}")
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_loader.dataset)
        scheduler.step()
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss}")
        # Learnig rate:{scheduler.get_last_lr()}

# Call the function to train the model
train_model(model=model, criterion=criterion, optimizer=optimizer, device=device)

Epoch [1/100], Train Loss: 0.1039591688717957
Epoch [2/100], Train Loss: 0.05634422694733405
Epoch [3/100], Train Loss: 0.04749141339188702
Epoch [4/100], Train Loss: 0.038118366802517525
Epoch [5/100], Train Loss: 0.03470186113698002
Epoch [6/100], Train Loss: 0.02287186611112238
Epoch [7/100], Train Loss: 0.019372589614054518
Epoch [8/100], Train Loss: 0.019996505938863207
Epoch [9/100], Train Loss: 0.015329804927701852
Epoch [10/100], Train Loss: 0.013279942094526587
Epoch [11/100], Train Loss: 0.009402560515507563
Epoch [12/100], Train Loss: 0.0065970286877045925
Epoch [13/100], Train Loss: 0.005727700327962365
Epoch [14/100], Train Loss: 0.0069607318473075335
Epoch [15/100], Train Loss: 0.005836522982518309
Epoch [16/100], Train Loss: 0.0021757292596544645
Epoch [17/100], Train Loss: 0.0031010039632580885
Epoch [18/100], Train Loss: 0.0021728620168381327
Epoch [19/100], Train Loss: 0.001286242522934424
Epoch [20/100], Train Loss: 0.0019157075513439176
Epoch [21/100], Train Loss: 0

In [54]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
          inputs, labels = inputs.to(device), labels.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs, 1)
          _, labels_indices = torch.max(labels, 1)  # Convert one-hot labels to indices
          total += labels.size(0)
          correct += (predicted == labels_indices).sum().item()
    accuracy = correct / total
    print(f"Total accurate predictions: {correct} out of {total}")
    print(f"Accuracy on test set: {accuracy:.4f}")

test_model(model=model,test_loader=test_loader)

# Save the model
torch.save(model.state_dict(), 'code/asl_cnn_model.pth')

Total accurate predictions: 2530 out of 2700
Accuracy on test set: 0.9370


In [45]:
model.load_state_dict(torch.load('code/asl_cnn_model.pth'))
model.eval()


ConvNet(
  (conv1): Conv2d(3, 27, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(27, 27, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=972, out_features=27, bias=True)
)